In [1]:
import tkinter as tk
import customtkinter as ctk
from tkinter import messagebox, filedialog
from RunLength import RunLengthEncoder
from Arithmetic import ArithmeticEncoder

In [2]:
class TextEncodingApp(ctk.CTk):
    def __init__(self):
        super().__init__()

        # window properties
        self.title("Compression Techniques")
        self.iconbitmap("images/icon.ico")
        ctk.set_appearance_mode("Light")
        ctk.set_default_color_theme("green")

        # Make the App start on the center of the screen
        screen_width = self.winfo_screenwidth()
        screen_height = self.winfo_screenheight()
        self.x = (screen_width / 2) - (1280 / 2)
        self.y = (screen_height / 2) - (720 / 2)
        self.geometry(f"1280x720+{int(self.x)}+{int(self.y)}")

        self.title = ctk.CTkLabel(self, text = "Text Compression", text_color = "#6527BE", font = ("Arial Bold", 24))
        self.title.pack(padx = 10)

        self.theme_button = ctk.CTkButton(self, text="Change Theme", corner_radius=5, command=self.change_theme)
        self.theme_button.pack(side="top", anchor="ne", padx=10)

        self.input_text = ctk.CTkTextbox(self, width=screen_width/4, height=100)
        self.input_text.pack(pady = 10)

        buttons_frame = ctk.CTkFrame(self, fg_color="transparent")
        buttons_frame.pack(pady=10)

        self.encode_button = ctk.CTkButton(buttons_frame, text="Encode", corner_radius=5, command=self.encode)
        self.encode_button.pack(side="left", padx=(10, 0), pady=5)

        self.or_label = ctk.CTkLabel(buttons_frame, text="OR", font=("Arial", 12, "bold"))
        self.or_label.pack(side="left", padx=(10, 10), pady=5)

        self.file_picker_button = ctk.CTkButton(buttons_frame, text="Browse a File", corner_radius=5, command=lambda: self.encode(isFile=True))
        self.file_picker_button.pack(side="left", padx=(0, 10), pady=5)

        frames = ctk.CTkFrame(self, fg_color="transparent")
        frames.pack(fill="both", expand=True, pady=10)

        self.run_length_frame = ctk.CTkScrollableFrame(frames, corner_radius=12, fg_color="#76ABAE")
        self.run_length_frame.pack(anchor='n', side = 'left', fill="both", expand=True, pady = 10, padx = 15)
        ctk.CTkLabel(self.run_length_frame, text='Enter or Upload a text to start ', text_color="#31363F", font = ("Arial Bold", 12)).pack(pady = 150)
     
        self.huffman_frame = ctk.CTkScrollableFrame(frames, corner_radius=12, fg_color="#ECB159")
        self.huffman_frame.pack(anchor='n', side = 'left', fill="both", expand=True, pady = 10, padx = 15)
        ctk.CTkLabel(self.huffman_frame, text='Enter or Upload a text to start ', text_color="#31363F", font = ("Arial Bold", 12)).pack(pady = 150)

        self.arithmetic_frame = ctk.CTkScrollableFrame(frames, corner_radius=12, fg_color="#A4CE95")
        self.arithmetic_frame.pack(anchor='n', side = 'left', fill="both", expand=True, pady = 10, padx = 15)
        ctk.CTkLabel(self.arithmetic_frame, text='Enter or Upload a text to start ', text_color="#31363F", font = ("Arial Bold", 12)).pack(pady = 150)

        self.golomb_frame = ctk.CTkScrollableFrame(frames, corner_radius=12, fg_color="#90D26D")
        self.golomb_frame.pack(anchor='n', side = 'left', fill="both", expand=True, pady = 10, padx = 15)
        ctk.CTkLabel(self.golomb_frame, text='Enter or Upload a text to start ', text_color="#31363F", font = ("Arial Bold", 12)).pack(pady = 150)

        self.LZW_frame = ctk.CTkScrollableFrame(frames, corner_radius=12, fg_color="#7AA2E3")
        self.LZW_frame.pack(anchor='n', side = 'left', fill="both", expand=True, pady = 10, padx = 15)
        ctk.CTkLabel(self.LZW_frame, text='Enter or Upload a text to start ', text_color="#31363F", font = ("Arial Bold", 12)).pack(pady = 150)

        self.recommendation_frame = ctk.CTkFrame(self, fg_color="#A6CF98")
        self.recommendation_frame.pack(fill="both", expand=True,padx=15, pady=10)

        ctk.CTkLabel(self.recommendation_frame,text='Recommendation', text_color="#6527BE", font = ("Arial Bold", 18)).pack(pady = 10)
        ctk.CTkLabel(self.recommendation_frame,text='The best compression technique for this text is: ', text_color="#31363F", font = ("Arial Bold", 12)).pack(anchor="center", pady = 10)

    def display_info(self, frame, frame_title, bits_before, bits_after, compression_ratio, entropy, average_length, efficiency, probability):
        frame_label = ctk.CTkLabel(frame, text=frame_title, font=("Arial", 20, "bold"))
        frame_label.pack(pady=5)

        bits_before_label = ctk.CTkLabel(frame, text=f'bits(before): {bits_before}', font=("Arial", 12, "bold"))
        bits_before_label.pack(anchor='w', padx=10,pady=5)

        bits_after_label = ctk.CTkLabel(frame, text=f'bits(after): {bits_after}', font=("Arial", 12, "bold"))
        bits_after_label.pack(anchor='w', padx=10,pady=5)

        bits_entropy_label = ctk.CTkLabel(frame, text=f'Entropy: {entropy}', font=("Arial", 12, "bold"))
        bits_entropy_label.pack(anchor='w', padx=10,pady=5)

        bits_average_length_label = ctk.CTkLabel(frame, text=f'Average Length: {average_length}', font=("Arial", 12, "bold"))
        bits_average_length_label.pack(anchor='w', padx=10,pady=5)

        bits_compression_ratio_label = ctk.CTkLabel(frame, text=f'Compression Ratio: {compression_ratio}%', font=("Arial", 12, "bold"))
        bits_compression_ratio_label.pack(anchor='w', padx=10,pady=5)

        bits_efficiency_label = ctk.CTkLabel(frame, text=f'Efficiency: {efficiency}%', font=("Arial", 12, "bold"))
        bits_efficiency_label.pack(anchor='w', padx=10,pady=5)
        
        bits_probability_label = ctk.CTkLabel(frame, text=f'Probabilities: ', font=("Arial", 12, "bold"))
        bits_probability_label.pack(anchor='w', padx=10,pady=5)

        for char, probability in probability.items():
            if char == ' ':
                char = 'Space'
            if char == '\n':
                char = 'New Line'
            probability_label = ctk.CTkLabel(frame, text=f'{char}: {round(probability, 2)}', font=("Arial", 12))
            probability_label.pack(anchor='w', padx=30)

    def get_path(self):
        filepath = filedialog.askopenfilename(initialdir=r"E:\Faculty Material\Third Level\Second Term\Data Compression\Final Project", title="Select A File", filetypes=(("Text Files", "*.txt"),("all files", "*.*")))
        return filepath
    
    def encode(self, isFile=False):
        frames = [self.run_length_frame, self.huffman_frame, self.arithmetic_frame, self.golomb_frame, self.LZW_frame]
        
        if isFile:
            file_path = self.get_path()
            if not file_path:
                messagebox.showerror("File Error", "No file selected.")
                return
            try:
                text = open(file_path, 'r').read()
            except Exception as e:
                messagebox.showerror("File Error", f"Error reading file")
                return
            if not text:
                messagebox.showerror("Empty File", "The selected file is empty.")
                return
            
            self.input_text.delete('1.0', 'end')
            self.input_text.insert('1.0', text)
        else:
            text = self.input_text.get('1.0', 'end-1c')
            if not text:
                messagebox.showerror("Empty Text", "Please enter a text to start encoding.")
                return

       
        rle = RunLengthEncoder.RLE_encoding(text)
        Arithmetic = ArithmeticEncoder.Arithmetic_encoding(text)


        # Clear all frames
        for frame in frames:
            for widget in frame.winfo_children():
                widget.destroy()

        # Update Run-Length frame
        self.display_info(self.run_length_frame, 'Run-Length', rle["bits_before"], rle["bits_after"], rle["compression ratio (%)"], rle["entropy"] ,
                        rle["average_length"], rle["efficiency"], rle['probabilities'])

        # Update Huffman frame
        self.display_info(self.huffman_frame, 'Huffman', 'bits_before', 'bits_after', 'compression_ratio', 'entropy',
                        'average_length', 'efficiency', {})

        # Update Arithmetic frame
        self.display_info(self.arithmetic_frame, 'Arithmetic',
                           Arithmetic['bits_before'], Arithmetic['bits_after'], Arithmetic["compression ratio (%)"], Arithmetic['entropy'],
                        Arithmetic['average_length'], Arithmetic['efficiency'], Arithmetic['probabilities'])

        # Update Golomb frame
        self.display_info(self.golomb_frame, 'Golomb', 'bits_before', 'bits_after', 'compression_ratio', 'entropy',
                        'average_length', 'efficiency', {})

        # Update LZW frame
        self.display_info(self.LZW_frame, 'LZW', 'bits_before', 'bits_after', 'compression_ratio', 'entropy',
                        'average_length', 'efficiency', {})

    def change_theme(self):
        if ctk.get_appearance_mode() == "Light":
            ctk.set_appearance_mode("Dark")
        else:
            ctk.set_appearance_mode("Light")

        
if __name__ == "__main__":
    app = TextEncodingApp()
    app.mainloop()
